# MN2023_A2: Eigenvalues #
Outline
1. Power method
2. QR Decomposition method
3. Singular value decomposition (SVD) method

## Power Method ##
Find dominant eigenvalues and its corresponding eigenvector of a matrix.

In [17]:
import numpy as np
import math

In [18]:
def matrix_vector_multiplication(a, x):
    result = np.zeros_like(x)
    for i in range(len(a)):
        for j in range(len(a)):
            result[i] += a[i,j] * x[j]
    return result

def norm(vector):
    norm = 0
    for x in vector:
        norm += np.power(x, 2)
    return np.power(norm, 0.5)

In [19]:
def power_method(a, maxiter=100, init_vector=None):
    if init_vector is None:
        n = len(a)
        init_vector = np.random.rand(n)
    
    for i in range(maxiter):
        
        # a.init_vector = res
        res = matrix_vector_multiplication(a, init_vector)

        # calculate norm for res
        res_norm = norm(res)

        # init_vector = normalized version of res
        init_vector = [x / res_norm for x in res]

        # max eigen = init_vector.res
        eigenvalue = np.dot(init_vector, res)
    
    print("Maximum eigenvalue: ", eigenvalue)
    print("Maximum eigenvector: ", init_vector)
    return eigenvalue, init_vector


In [20]:
powerTest = np.array([[2,-3,0],[2,-5,0],[0,0,3]])
power_method(powerTest)

Maximum eigenvalue:  4.0
Maximum eigenvector:  [0.447213595499958, 0.8944271909999159, 7.882823541404979e-13]


(4.0, [0.447213595499958, 0.8944271909999159, 7.882823541404979e-13])

## QR Decomposition Method ##
The code below is done accordingly to Berkeley's Python Numerical Method

In [21]:
def qr_decomposition_berkeley(A, max_iter = 100):
    A = A.astype(float)
    for i in range(max_iter):
        Q, R = np.linalg.qr(A)
        A = np.dot (R, Q)
    
    eigenvalues = np.diag(A)
    print("A: ")
    print(A)
    print("Eigenvalues: ", eigenvalues)

qr_decomposition_berkeley(np.array([
        [2, 1, 2],
        [1, 3, 4],
        [2, 2, 1]
    ]))

A: 
[[ 6.02911192e+00 -1.04289534e+00 -1.47753946e+00]
 [-1.65631866e-64 -4.62682475e-01 -9.16964963e-01]
 [-1.19106498e-64 -1.77092424e+00  4.33570555e-01]]
Eigenvalues:  [ 6.02911192 -0.46268247  0.43357055]


The code below is done accordingly by using Gram-Schmidt process

In [1]:
def norm(vec):
    norm = 0
    for i in range(len(vec)):
        norm += math.pow(vec[i],2)
    return norm

def cur_vec(a, i):
    vec = a[0:len(a),i]
    return vec

def gs(a):
    n = len(a)
    # initialise final e vector
    e = np.zeros((n, n))

    vec = cur_vec(a,0) 
    temp = norm(vec)
    
    for i in range(n):
        e[i,0]=vec[i]/math.sqrt(temp)

    for k in range(1, n):
        vec = cur_vec(a,k)
        temp_res = 0
        for l in range(0,k):
            vec_e = cur_vec(e,l)
            temp_res += np.dot(vec,vec_e)*vec_e
            
        numerator = vec - temp_res
        for m in range(n):
            e[m,k]=numerator[m]/math.sqrt(norm(numerator))

    return e

def row_op(a):
    A = np.copy(a)
    n=len(A)
    # OBE
    # standard case
    for i in range(n):
        for j in range(i+1, n):
            k = -1 * A[j,i] / A[i,i]
            A[j,i:n] = k * A[i,i:n] + A[j,i:n]
    return A

def qr_decomp(a, max_iter=100):

    A = np.copy(a)
    n=len(A)

    for i in range(max_iter):
        Q = gs(A)
        R = np.matmul(Q.T, A)
        A = np.matmul(R,Q)

    res = np.zeros(n)
    for i in range(n):
        res[i] = A[i,i]

    return res

## SVD method
Singular Value Decomposition

In [24]:
def gs2(a):
    # initialise final e vector
    SIZE = len(a)
    e = np.zeros((SIZE, SIZE))

    # k = 1
    # grab current vector
    vec = cur_vec(a, 0)

    # calculate norm
    temp = norm(vec)

    # find e_1
    for i in range(SIZE):
        e[i, 0] = vec[i] / np.sqrt(temp)

    # k > 1
    for k in range(1, SIZE):
        vec = cur_vec(a, k)
        temp_res = 0

        for l in range(0, k):
            vec_e = cur_vec(e, l)
            temp_res += np.dot(vec, vec_e) * vec_e
        numerator = vec - temp_res

        for m in range(SIZE):
            e[m, k] = numerator[m] / np.sqrt(norm(numerator))

    return e

def qr(a, iteration=16):
    # this function uses QR decomposition
    a = np.copy(a)
    n = len(a)
    val = np.zeros((n, n))

    # inject 1 to columns without a leading 1
    for k in range(n):
        cvc = a[:, k]
        sum = 0
        for i in range(len(cvc)):
            sum += cvc[i]
        if sum > 0:
            val[k] = cvc
        elif sum == 0:
            cvc[k] = 1
            val[k] = cvc
    val = val.T
    # print("val=", val)

    vec = np.eye(n)

    for _ in range(iteration):
        Q = gs2(val)
        vec = vec @ np.copy(Q)
        R = Q.T @ val  # A_k = Q_k * R_k
        val = R @ Q  # A_(k+1) = R_k * Q_k

    return val.diagonal(), vec


def svd_method(a):
    A = np.copy(a)

    SL = A @ A.T
    SR = A.T @ A

    val_L, vec_L = qr(SL)
    val_R, vec_R = qr(SR)

    U = np.zeros((len(SL), len(SL)))
    for k in range(len(SL)):
        cvc = vec_L[:, k]

        norm = 0
        for i in range(len(cvc)):
            norm += cvc[i] ** 2
        norm = np.sqrt(norm)

        ncvc = np.zeros(len(SL))
        for i in range(len(SL)):
            ncvc[i] = cvc[i] / norm

        U[k] = ncvc
    U = U.T 

    V = np.zeros((len(SR), len(SR)))
    for k in range(len(SR)):
        cvc = vec_R[:, k]

        norm = 0
        for i in range(len(cvc)):
            norm += cvc[i] ** 2
        norm = np.sqrt(norm)

        ncvc = np.zeros(len(SR))
        for i in range(len(SR)):
            ncvc[i] = cvc[i] / norm

        V[k] = ncvc

    size = np.shape(A)
    S = np.zeros(size)

    for i in range(min(size[0], size[1])):
        S[i, i] = np.sqrt(val_L[i])

    return U, S, V